### РОССИЙСКИЙ УНИВЕРСИТЕТ ДРУЖБЫ НАРОДОВ

#### Факультет физико-математических и естественных наук  
#### Кафедра информационных технологий 


## ОТЧЕТ ПО ЛАБОРАТОРНОЙ РАБОТЕ № 2


### Дисциплина: Интеллектуальный анализ данных



##### Студент:  Липатникова Марина
##### Группа:   НФИбд-02-19



## Москва 2022
***


#### Задание на лабораторную работу №2



Для закрепленного за Вами варианта лабораторной работы:

    1. При помощи модуля sqlite3 откройте базу данных Instacart в файле instacart.db.

    2. Загрузите таблицы departments и products в датафреймы Pandas. При помощи запроса SELECT извлеките из таблицы order_products__train записи, соответствующие указанным в индивидуальном задании дню недели (поле order_dow таблицы orders) и коду департамента (поле department_id таблицы products) и загрузите в датафрейм Pandas. Определите количество строк в полученном датафрейме и определите количество товаров (стоолбец product_id) в транзакциях датафрейма.

    3. Определите пять наиболее популярных товаров в датафрейме транзакций и определите количество покупок (транзакций) этих товаров.

    4. Постройте транзакционную базу данных из полученного датафрейма, используя в качестве идентификатора транзакции столбец order_id, а в качестве названий товаров - поле product_name из датафрейма для таблицы products, соответствующее столбцу product_id. Найдите в транзакционной базе данных три транзакции с наибольшим количеством товаров и выведите их на экран.

    5. Постройте по транзакционной базе данных бинарную базу данных в формате датафрейма пакета mlxtend. По бинарной базе данных определите пять наиболее популярных товаров и определите количество покупок (транзакций) этих товаров.

    6. При помощи указанного в индивидуальном задании метода построения популярных наборов предметов постройте популярный набор предметов с минимальной поддержкой не менее 3, имеющий максимальную длину. При отсутствии таких наборов уменьшите поддержку до 2. В случае нехватки вычислительных ресурсов (слишком долгой работы программы) при построении популярных наборов предметов сокращайте число записей в наборе данных (например, делая выборку половины записей набора).

    7. Используя пакет mlxtend или реализацию на Python, постройте набор ассоциативных правил для полученного популярного наборов предметов. Используйте уровень достоверности (confidence), равный 0.6.

    8. Для построенного набора ассоциативных правил вычислите показатель (меру) оценки ассоциативных правил, указанную в индивидуальном задании, и определите ассоциативные правила с наилучшим значением показателя оценки.






# Индивидуальное задание



Лабораторная работа 2 – Вариант 13

Алгоритм: FPMax

День недели (поле order_dow таблицы orders): “1”

Код департамента (поле department_id таблицы products): “21”

Показатель оценки ассоциативных правил: лифт (lift) 


1.	При помощи модуля sqlite3 откройте базу данных Instacart в файле instacart.db.


Файл был переименован из-за случайной проблемы, а после забылось.

In [1]:
import numpy as np
import pandas as pd
import sqlite3

conn = sqlite3.connect('pp.db')
cursor = conn.cursor()



2.	Загрузите таблицы departments и products в датафреймы Pandas. При помощи запроса SELECT извлеките из таблицы order_products__train записи, соответствующие указанным в индивидуальном задании дню недели (поле order_dow таблицы orders) и коду департамента (поле department_id таблицы products) и загрузите в датафрейм Pandas. Определите количество строк в полученном датафрейме и определите количество товаров (стоолбец product_id) в транзакциях датафрейма.


In [2]:
df_dept = pd.read_sql_query("SELECT * FROM departments", conn)
df_dept


,department_id,department
0,1,frozen
1,2,other
2,3,bakery
3,4,produce
4,5,alcohol
5,6,international
6,7,beverages
7,8,pets
8,9,dry goods pasta
9,10,bulk


In [3]:
df_prod = pd.read_sql_query("SELECT * FROM products", conn)
df_prod


,product_id,product_name,aisle_id,department_id
0,1,Chocolate Sandwich Cookies,61,19
1,2,All-Seasons Salt,104,13
2,3,Robust Golden Unsweetened Oolong Tea,94,7
3,4,Smart Ones Classic Favorites Mini Rigatoni Wit...,38,1
4,5,Green Chile Anytime Sauce,5,13
...,...,...,...,...
49683,49684,"Vodka, Triple Distilled, Twist of Vanilla",124,5
49684,49685,En Croute Roast Hazelnut Cranberry,42,1
49685,49686,Artisan Baguette,112,3
49686,49687,Smartblend Healthy Metabolism Dry Cat Food,41,8


In [4]:
data = pd.read_sql_query("""
    SELECT opt.order_id, opt.product_id, opt.add_to_cart_order, opt.reordered
    FROM order_products__train as opt,orders as ord,products as prod
    WHERE opt.product_id=prod.product_id AND prod.department_id=21 AND opt.order_id=ord.order_id AND ord.order_dow=1
    """, conn)
data


,order_id,product_id,add_to_cart_order,reordered
0,2832847,38,6,0
1,2874700,491,25,0
2,430374,624,11,1
3,1578900,768,6,0
4,2975822,768,3,0
...,...,...,...,...
1059,1125109,49484,5,1
1060,1684053,49484,1,1
1061,1949159,49484,19,1
1062,736717,49484,2,0



Количество строк = 1064.

In [5]:
count = len(data["product_id"].unique())
count

336


Количество товаров = 336.


3.	Определите пять наиболее популярных товаров в датафрейме транзакций и определите количество покупок (транзакций) этих товаров.


In [6]:
for row in cursor.execute("""
    select count(opt.order_id) as orders, prod.product_name as popular_product
    from order_products__train opt, products prod, orders as ord
    where opt.product_id=prod.product_id AND prod.department_id=21 AND opt.order_id=ord.order_id AND ord.order_dow=1
    group by popular_product 
    order by orders desc 
    limit 5
    """):
    print(row)
    

(120, 'Organic Riced Cauliflower')
(36, 'Peanut Butter Ice Cream Cup')
(33, 'Organic Celery Bunch')
(33, 'Natural Lemon Flavored Sparkling Water')
(27, 'Cookie Dough Ice Cream Chocolate Chip')


4.	Постройте транзакционную базу данных из полученного датафрейма, используя в качестве идентификатора транзакции столбец order_id, а в качестве названий товаров - поле product_name из датафрейма для таблицы products, соответствующее столбцу product_id. Найдите в транзакционной базе данных три транзакции с наибольшим количеством товаров и выведите их на экран.

In [7]:
for row in cursor.execute("""
    select count(prod.product_id) as productes, opt.order_id as max_prod
    from order_products__train opt, products prod, orders as ord
    where opt.product_id=prod.product_id AND prod.department_id=21 AND opt.order_id=ord.order_id AND ord.order_dow=1
    group by max_prod
    order by productes desc 
    limit 3
    """):
    print(row)
    

(4, '991562')
(4, '727951')
(4, '3041233')


5.	Постройте по транзакционной базе данных бинарную базу данных в формате датафрейма пакета mlxtend. По бинарной базе данных определите пять наиболее популярных товаров и определите количество покупок (транзакций) этих товаров.

In [8]:
data2 = pd.read_sql_query("""
    SELECT opt.order_id, prod.product_name
    FROM order_products__train as opt,orders as ord,products as prod
    WHERE opt.product_id=prod.product_id AND prod.department_id=21 AND opt.order_id=ord.order_id AND ord.order_dow=1
    """, conn)
data2


,order_id,product_name
0,2832847,Ultra Antibacterial Dish Liquid
1,2874700,Pompelmo Water
2,430374,Chub Chorizo
3,1578900,Nacho Grain-Free Tortilla Chips
4,2975822,Nacho Grain-Free Tortilla Chips
...,...,...
1059,1125109,Grain Free Cassava & Chia Tortillas
1060,1684053,Grain Free Cassava & Chia Tortillas
1061,1949159,Grain Free Cassava & Chia Tortillas
1062,736717,Grain Free Cassava & Chia Tortillas


In [9]:
list = data2.values.tolist()


In [10]:
list_id = []
for i in list:
    if i[0] in list_id:
        continue
    else:
        list_id.append(i[0])
        

In [11]:
list1 = []
list2 = []
for i in list_id:
    list1 = []
    for j in list:
        if i == j[0]:
            list1.append(j[1])
    list2.append(list1)
list2


[['Ultra Antibacterial Dish Liquid'],
 ['Pompelmo Water'],
 ['Chub Chorizo'],
 ['Nacho Grain-Free Tortilla Chips'],
 ['Nacho Grain-Free Tortilla Chips', 'Lime Grain-Free Tortilla Chips'],
 ['Nacho Grain-Free Tortilla Chips'],
 ['Red Sweet Mini Pepper'],
 ['White Apple Cauliflower & Leek Organic Baby Food'],
 ['White Apple Cauliflower & Leek Organic Baby Food',
  'Eat Your Colors Purples Puree Baby Food',
  'Organic Nondairy Lemon Cashew Yogurt',
  'Eat Your Colors Reds Puree Baby Food'],
 ['Onion Crispbread'],
 ['Paso Robles Cabernet Sauvignon'],
 ['Kombucha Beet'],
 ['Lime Grain-Free Tortilla Chips'],
 ['Lime Grain-Free Tortilla Chips', 'Organic Blue Corn Tortilla Dippers'],
 ['Lime Grain-Free Tortilla Chips'],
 ['Lime Grain-Free Tortilla Chips'],
 ['Lime Grain-Free Tortilla Chips', 'Artichoke'],
 ['Organic Chopped BBQ Ranch Salad Kit'],
 ['Organic Chopped BBQ Ranch Salad Kit'],
 ['Organic Chopped BBQ Ranch Salad Kit', 'Organic Asian Chopped Salad Kit'],
 ['Organic Chopped BBQ Ranch S

In [12]:
from mlxtend.preprocessing import TransactionEncoder

te = TransactionEncoder()
dataset_bin = te.fit(list2).transform(list2)
data3 = pd.DataFrame(dataset_bin, columns=te.columns_)
data3


,100% Cauliflower & Sweet Potato Riced Veggies,100% Grass-Fed No-Grain Strawberry Yogurt,100% Natural Ginger Ale,100% Orange Peach Mango Juice,100% Probiotics Pineapple Mango Juice,100% Pure Organic Coconut Water,12 G. Protein Bar Coffee Chocolate,2% Milkfat Lowfat Cottage Cheese Large Curd,3 Day Cleanse Total Body Reset Advanced Herbal Formula Capsules,3 Tri Colored Organic Peppers,...,Water Filtration Pitcher Replacement Filter,White Apple Cauliflower & Leek Organic Baby Food,"White Bean Crunch, Mac N' Cheese",Whole Milk Pear Spinach & Mango Yogurt Pouch,Whole Milk Pure Honey Pear,Whole Milk Strawberry Smoothies,Whole Milk Yogurt Organic Indonesian Vanilla Bean,Yam,Yellow Onion Bag,Zebra Popcorn
0,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
1,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
2,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
3,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
4,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
893,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
894,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
895,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
896,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False


По бинарной базе данных определите пять наиболее популярных товаров и определите количество покупок (транзакций) этих товаров.

In [13]:
data3.sum(axis=0).nlargest(5)


Organic Riced Cauliflower                 120
Peanut Butter Ice Cream Cup                36
Natural Lemon Flavored Sparkling Water     33
Organic Celery Bunch                       33
Cookie Dough Ice Cream Chocolate Chip      27
dtype: int64

6.	При помощи указанного в индивидуальном задании метода построения популярных наборов предметов постройте популярный набор предметов с минимальной поддержкой не менее 3, имеющий максимальную длину. При отсутствии таких наборов уменьшите поддержку до 2. В случае нехватки вычислительных ресурсов (слишком долгой работы программы) при построении популярных наборов предметов сокращайте число записей в наборе данных (например, делая выборку половины записей набора).

Чтобы выводило несколько строчек с более 2-х продуктов в наборе: min_support=0.002

In [14]:
from mlxtend.frequent_patterns import fpmax

itemsets_fpm = fpmax(data3, min_support=0.002, use_colnames=True)
itemsets_fpm


,support,itemsets
0,0.002227,(Organic Baby Carrot Bag)
1,0.002227,(Raise The Roots Kombucha)
2,0.002227,(Natural Bliss Coconut Milk Coffee Creamer)
3,0.002227,"(Protein Bar Maple Sea Salt, Protein Bar Mint ..."
4,0.002227,(Platinum ActionPacs Dishwasher Detergent)
...,...,...
115,0.003341,"(Peanut Butter Ice Cream Cup, S'mores Ice Crea..."
116,0.002227,"(Peanut Butter Ice Cream Cup, Cookie Dough Ice..."
117,0.004454,"(Organic Celery Bunch, Organic Riced Cauliflower)"
118,0.036748,(Natural Lemon Flavored Sparkling Water)


7.	Используя пакет mlxtend или реализацию на Python, постройте набор ассоциативных правил для полученного популярного наборов предметов. Используйте уровень достоверности (confidence), равный 0.6.

Набор ассоциативных правил для полученного популярных наборов предметов не вывело ничего.

In [15]:
from mlxtend.frequent_patterns import association_rules

as_rules = association_rules(itemsets_fpm, metric="confidence", min_threshold=0.6, support_only=True)
as_rules


,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction


И даже с уменьшением min_threshold выдает странный результат.

In [17]:
from mlxtend.frequent_patterns import association_rules

as_rules = association_rules(itemsets_fpm, metric="confidence", min_threshold=0.003, support_only=True)
as_rules


,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction
0,(Organic Grassmilk Yogurt Wild Blueberry),(100% Grass-Fed No-Grain Strawberry Yogurt),NaN,NaN,0.003341,NaN,NaN,NaN,NaN
1,(100% Grass-Fed No-Grain Strawberry Yogurt),(Organic Grassmilk Yogurt Wild Blueberry),NaN,NaN,0.003341,NaN,NaN,NaN,NaN
2,(Organic Mango Yogurt),(Organic Whole Milk Washington Black Cherry Yo...,NaN,NaN,0.005568,NaN,NaN,NaN,NaN
3,(Organic Whole Milk Washington Black Cherry Yo...,(Organic Mango Yogurt),NaN,NaN,0.005568,NaN,NaN,NaN,NaN
4,"(Peanut Butter Ice Cream Cup, S'mores Ice Cream)",(Oatneal Cookie Ice Cream),NaN,NaN,0.003341,NaN,NaN,NaN,NaN
5,"(Peanut Butter Ice Cream Cup, Oatneal Cookie I...",(S'mores Ice Cream),NaN,NaN,0.003341,NaN,NaN,NaN,NaN
6,"(S'mores Ice Cream, Oatneal Cookie Ice Cream)",(Peanut Butter Ice Cream Cup),NaN,NaN,0.003341,NaN,NaN,NaN,NaN
7,(Peanut Butter Ice Cream Cup),"(S'mores Ice Cream, Oatneal Cookie Ice Cream)",NaN,NaN,0.003341,NaN,NaN,NaN,NaN
8,(S'mores Ice Cream),"(Peanut Butter Ice Cream Cup, Oatneal Cookie I...",NaN,NaN,0.003341,NaN,NaN,NaN,NaN
9,(Oatneal Cookie Ice Cream),"(Peanut Butter Ice Cream Cup, S'mores Ice Cream)",NaN,NaN,0.003341,NaN,NaN,NaN,NaN


Поэтому было решено сделать с другими алгоритмами и теми же характеристиками.

In [19]:
from mlxtend.frequent_patterns import fpgrowth

itemsets_fpg = fpgrowth(data3, min_support=0.002,use_colnames=True)
itemsets_fpg


,support,itemsets
0,0.003341,(Nacho Grain-Free Tortilla Chips)
1,0.006682,(Lime Grain-Free Tortilla Chips)
2,0.002227,(White Apple Cauliflower & Leek Organic Baby F...
3,0.004454,(Eat Your Colors Reds Puree Baby Food)
4,0.003341,(Eat Your Colors Purples Puree Baby Food)
...,...,...
150,0.002227,"(Organic Grassmilk Yogurt Wild Blueberry, Orga..."
151,0.002227,"(Protein Bar Maple Sea Salt, Protein Bar Mint ..."
152,0.002227,"(Organic Vanilla Grassmilk Yogurt, 100% Grass-..."
153,0.003341,"(Organic Grassmilk Yogurt Wild Blueberry, 100%..."


In [20]:
from mlxtend.frequent_patterns import apriori

itemsets_apr = apriori(data3, min_support=0.002,use_colnames=True)
itemsets_apr


,support,itemsets
0,0.004454,(100% Grass-Fed No-Grain Strawberry Yogurt)
1,0.002227,(12 G. Protein Bar Coffee Chocolate)
2,0.002227,(3 Day Cleanse Total Body Reset Advanced Herba...
3,0.002227,(4 Cheese Ravioli)
4,0.003341,(Almond Milk Vanilla Yogurt)
...,...,...
150,0.002227,"(Vanilla Whole Milk Drinkable Yogurt, Strawber..."
151,0.002227,"(Cookie Dough Ice Cream Chocolate Chip, S'more..."
152,0.002227,"(Peanut Butter Ice Cream Cup, Cookie Dough Ice..."
153,0.003341,"(Peanut Butter Ice Cream Cup, S'mores Ice Crea..."


Получилась более приятная картина, причем оба алгоритма совпали результатами.

In [21]:
from mlxtend.frequent_patterns import association_rules

as_rules1 = association_rules(itemsets_apr, metric="confidence", min_threshold=0.6)
as_rules1


,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction
0,(Organic Grassmilk Yogurt Wild Blueberry),(100% Grass-Fed No-Grain Strawberry Yogurt),0.004454,0.004454,0.003341,0.750000,168.375000,0.003321,3.982183
1,(100% Grass-Fed No-Grain Strawberry Yogurt),(Organic Grassmilk Yogurt Wild Blueberry),0.004454,0.004454,0.003341,0.750000,168.375000,0.003321,3.982183
2,(Pizza Mac & Cheese),(General Tsao's Tofu),0.002227,0.005568,0.002227,1.000000,179.600000,0.002215,inf
3,(Protein Bar Mint Chocolate),(Protein Bar Maple Sea Salt),0.002227,0.011136,0.002227,1.000000,89.800000,0.002202,inf
4,(Vanilla Whole Milk Drinkable Yogurt),(Strawberry Whole Milk Drinkable Yogurt),0.002227,0.007795,0.002227,1.000000,128.285714,0.002210,inf
5,"(S'mores Ice Cream, Oatneal Cookie Ice Cream)",(Peanut Butter Ice Cream Cup),0.005568,0.040089,0.003341,0.600000,14.966667,0.003118,2.399777
6,"(Organic Cashew Nondairy Vanilla Yogurt, Organ...",(Organic Nondairy Strawberry Cashew Yogurt),0.003341,0.015590,0.002227,0.666667,42.761905,0.002175,2.953229


In [22]:
from mlxtend.frequent_patterns import association_rules

as_rules2 = association_rules(itemsets_apr, metric="confidence", min_threshold=0.6)
as_rules2


,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction
0,(Organic Grassmilk Yogurt Wild Blueberry),(100% Grass-Fed No-Grain Strawberry Yogurt),0.004454,0.004454,0.003341,0.750000,168.375000,0.003321,3.982183
1,(100% Grass-Fed No-Grain Strawberry Yogurt),(Organic Grassmilk Yogurt Wild Blueberry),0.004454,0.004454,0.003341,0.750000,168.375000,0.003321,3.982183
2,(Pizza Mac & Cheese),(General Tsao's Tofu),0.002227,0.005568,0.002227,1.000000,179.600000,0.002215,inf
3,(Protein Bar Mint Chocolate),(Protein Bar Maple Sea Salt),0.002227,0.011136,0.002227,1.000000,89.800000,0.002202,inf
4,(Vanilla Whole Milk Drinkable Yogurt),(Strawberry Whole Milk Drinkable Yogurt),0.002227,0.007795,0.002227,1.000000,128.285714,0.002210,inf
5,"(S'mores Ice Cream, Oatneal Cookie Ice Cream)",(Peanut Butter Ice Cream Cup),0.005568,0.040089,0.003341,0.600000,14.966667,0.003118,2.399777
6,"(Organic Cashew Nondairy Vanilla Yogurt, Organ...",(Organic Nondairy Strawberry Cashew Yogurt),0.003341,0.015590,0.002227,0.666667,42.761905,0.002175,2.953229


8.	Для построенного набора ассоциативных правил вычислите показатель (меру) оценки ассоциативных правил, указанную в индивидуальном задании, и определите ассоциативные правила с наилучшим значением показателя оценки.

In [23]:
as_rules = as_rules.sort_values(by='lift', ascending=False)
as_rules


,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction
0,(Organic Grassmilk Yogurt Wild Blueberry),(100% Grass-Fed No-Grain Strawberry Yogurt),NaN,NaN,0.003341,NaN,NaN,NaN,NaN
1,(100% Grass-Fed No-Grain Strawberry Yogurt),(Organic Grassmilk Yogurt Wild Blueberry),NaN,NaN,0.003341,NaN,NaN,NaN,NaN
2,(Organic Mango Yogurt),(Organic Whole Milk Washington Black Cherry Yo...,NaN,NaN,0.005568,NaN,NaN,NaN,NaN
3,(Organic Whole Milk Washington Black Cherry Yo...,(Organic Mango Yogurt),NaN,NaN,0.005568,NaN,NaN,NaN,NaN
4,"(Peanut Butter Ice Cream Cup, S'mores Ice Cream)",(Oatneal Cookie Ice Cream),NaN,NaN,0.003341,NaN,NaN,NaN,NaN
5,"(Peanut Butter Ice Cream Cup, Oatneal Cookie I...",(S'mores Ice Cream),NaN,NaN,0.003341,NaN,NaN,NaN,NaN
6,"(S'mores Ice Cream, Oatneal Cookie Ice Cream)",(Peanut Butter Ice Cream Cup),NaN,NaN,0.003341,NaN,NaN,NaN,NaN
7,(Peanut Butter Ice Cream Cup),"(S'mores Ice Cream, Oatneal Cookie Ice Cream)",NaN,NaN,0.003341,NaN,NaN,NaN,NaN
8,(S'mores Ice Cream),"(Peanut Butter Ice Cream Cup, Oatneal Cookie I...",NaN,NaN,0.003341,NaN,NaN,NaN,NaN
9,(Oatneal Cookie Ice Cream),"(Peanut Butter Ice Cream Cup, S'mores Ice Cream)",NaN,NaN,0.003341,NaN,NaN,NaN,NaN


In [24]:
as_rules1 = as_rules1.sort_values(by='lift', ascending=False)
as_rules1


,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction
2,(Pizza Mac & Cheese),(General Tsao's Tofu),0.002227,0.005568,0.002227,1.000000,179.600000,0.002215,inf
0,(Organic Grassmilk Yogurt Wild Blueberry),(100% Grass-Fed No-Grain Strawberry Yogurt),0.004454,0.004454,0.003341,0.750000,168.375000,0.003321,3.982183
1,(100% Grass-Fed No-Grain Strawberry Yogurt),(Organic Grassmilk Yogurt Wild Blueberry),0.004454,0.004454,0.003341,0.750000,168.375000,0.003321,3.982183
4,(Vanilla Whole Milk Drinkable Yogurt),(Strawberry Whole Milk Drinkable Yogurt),0.002227,0.007795,0.002227,1.000000,128.285714,0.002210,inf
3,(Protein Bar Mint Chocolate),(Protein Bar Maple Sea Salt),0.002227,0.011136,0.002227,1.000000,89.800000,0.002202,inf
6,"(Organic Cashew Nondairy Vanilla Yogurt, Organ...",(Organic Nondairy Strawberry Cashew Yogurt),0.003341,0.015590,0.002227,0.666667,42.761905,0.002175,2.953229
5,"(S'mores Ice Cream, Oatneal Cookie Ice Cream)",(Peanut Butter Ice Cream Cup),0.005568,0.040089,0.003341,0.600000,14.966667,0.003118,2.399777


In [25]:
as_rules2 = as_rules2.sort_values(by='lift', ascending=False)
as_rules2


,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction
2,(Pizza Mac & Cheese),(General Tsao's Tofu),0.002227,0.005568,0.002227,1.000000,179.600000,0.002215,inf
0,(Organic Grassmilk Yogurt Wild Blueberry),(100% Grass-Fed No-Grain Strawberry Yogurt),0.004454,0.004454,0.003341,0.750000,168.375000,0.003321,3.982183
1,(100% Grass-Fed No-Grain Strawberry Yogurt),(Organic Grassmilk Yogurt Wild Blueberry),0.004454,0.004454,0.003341,0.750000,168.375000,0.003321,3.982183
4,(Vanilla Whole Milk Drinkable Yogurt),(Strawberry Whole Milk Drinkable Yogurt),0.002227,0.007795,0.002227,1.000000,128.285714,0.002210,inf
3,(Protein Bar Mint Chocolate),(Protein Bar Maple Sea Salt),0.002227,0.011136,0.002227,1.000000,89.800000,0.002202,inf
6,"(Organic Cashew Nondairy Vanilla Yogurt, Organ...",(Organic Nondairy Strawberry Cashew Yogurt),0.003341,0.015590,0.002227,0.666667,42.761905,0.002175,2.953229
5,"(S'mores Ice Cream, Oatneal Cookie Ice Cream)",(Peanut Butter Ice Cream Cup),0.005568,0.040089,0.003341,0.600000,14.966667,0.003118,2.399777
